In [1]:
import os
import sys
import pandas as pd
import numpy as np


from PIL import Image
from tqdm import tqdm

In [2]:
import torch
from torch import nn,optim,Tensor
from torchvision import transforms
from torch.utils.data import DataLoader, random_split,Dataset

/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [30]:
# # Path to models

sys.path.append(os.path.abspath(".."))

from models.VGG13 import VGG13
from models.DataLoader import FaceDataset

In [4]:
SEED=42

In [31]:
# Training Variable


ROOT_PATH="/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning/data/raw/Images"
IN_CHANNELS=3
OUT_CHANNELS=3
BATCH_SIZE = 32
LEARNING_RATE=1e-4
EPOCHS=20

# Check is nvidia GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=(67.3686, 52.9727, 44.3268),std=(42.2854, 35.1020, 32.3488)),
    transforms.Resize((224,224),antialias=True)
])

# Initializing Datasets
dataset = FaceDataset(ROOT_PATH,transform)
generator = torch.Generator().manual_seed(SEED)

train_data,attack_data, test_data = random_split(dataset,[0.7,0.1,0.2],generator=generator)

print('Total number of dataset:',len(dataset))
print('Train                  :',len(train_data))
print('Attack                 :', len(attack_data))
print('Test                   :', len(test_data))


train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True, drop_last=True, generator=generator)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, generator=generator)


# Setting up the Model

model = VGG13(IN_CHANNELS,OUT_CHANNELS).to(device)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

Total number of dataset: 5000
Train                  : 3500
Attack                 : 500
Test                   : 1000


In [27]:
def checkpoint():
    ...

In [28]:
# inputs, targets = next(iter(train_loader)) 

for epoch in range(EPOCHS):
    
    model.train()
    correct=0
    total=0
    
    for idx, (inputs, targets) in tqdm(enumerate(train_loader),total=len(train_loader)):
        
        inputs = inputs.to(device)
        targets = targets.to(device)
    
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        # print(loss)
        loss.backward()
        optimizer.step()
    
        correct+=(outputs.argmax(dim=1) == targets).sum().item()
        total+=targets.size(0)
    
    print(f'Epoch [{epoch+1}/{EPOCHS}] | {correct}/{total} | Accuracy: {correct/total:.2f}')

### Found

In [ ]:
model = VGG13(IN_CHANNELS,OUT_CHANNELS).to(device)

train_loader = DataLoader(train_data, batch_size=1, shuffle=True, drop_last=True)

inputs, targets = next(iter(train_loader)) 


inputs = inputs.to(device)


# Get the base feature vector
base_vector = model.extract_features(inputs)

# calculate disruptor from gaussian noie




In [ ]:
base_vector.shape